In [3]:
import json

tags = json.load(open('../data/illustration2vec/mangaki_i2v.json'))
tags['1.jpg']

{'character': [],
 'copyright': [],
 'general': [['1girl', 0.5351361632347107],
  ['1boy', 0.5314481258392334],
  ['black hair', 0.4160270392894745],
  ['short hair', 0.4073851704597473],
  ['blue eyes', 0.2690363824367523],
  ['multiple boys', 0.16339154541492462],
  ['weapon', 0.16306668519973755],
  ['long hair', 0.16088248789310455],
  ['2boys', 0.15612725913524628],
  ['male', 0.1489870250225067],
  ['brown hair', 0.13320335745811462],
  ['scan', 0.11500567942857742],
  ['2girls', 0.10508275777101517]],
 'rating': [['safe', 0.9664666652679443],
  ['questionable', 0.0323437824845314],
  ['explicit', 0.00026572716888040304]]}

In [4]:
c = 0
for poster in tags:
    if tags[poster]['character']:
        print(poster, tags[poster]['character'])
        c += 1
        if c > 5:
            break

5307.jpg [['link', 0.5413832068443298]]
8000.jpg [['nagisa kaworu', 0.5120930075645447], ['ikari shinji', 0.2607155740261078]]
8436.jpg [['saigyouji yuyuko', 0.1193322166800499]]
5828.jpg [['ikari shinji', 0.13326214253902435]]
8071.jpg [['kousaka tamaki', 0.5505549311637878]]
11968.jpg [['pikachu', 0.25509053468704224]]


In [5]:
c = 0
for poster in tags:
    if tags[poster]['copyright']:
        print(poster, tags[poster]['copyright'])
        c += 1
        if c > 5:
            break

2361.jpg [['tengen toppa gurren lagann', 0.30961599946022034]]
5307.jpg [['the legend of zelda', 0.5240063071250916], ['twilight princess', 0.11857282370328903]]
8000.jpg [['neon genesis evangelion', 0.455232709646225], ['fate/zero', 0.1699470430612564], ['axis powers hetalia', 0.14517328143119812]]
279.jpg [['pokemon', 0.2236943542957306], ['original', 0.21127593517303467]]
5550.jpg [['idolmaster', 0.3257474899291992]]
12191.jpg [['original', 0.5091807246208191]]


In [6]:
all_tags = set()
for poster in tags:
    all_tags.update([tag for tag, _ in tags[poster]['general']])
nb_tags = len(all_tags)
nb_tags

503

In [7]:
list(all_tags)[:10]

['gloves',
 'penis',
 'outstretched arms',
 'military',
 'sash',
 'hoodie',
 'hand on hip',
 'masturbation',
 'spoilers',
 'multicolored hair']

In [8]:
from collections import Counter

tag_counter = Counter()
for poster in tags:
    for tag, weight in tags[poster]['general']:
        tag_counter[tag] += 1
tag_counter.most_common(5)

[('short hair', 9356),
 ('1girl', 8601),
 ('long hair', 8066),
 ('smile', 7922),
 ('brown hair', 7342)]

In [9]:
tag_counter.most_common(20)

[('short hair', 9356),
 ('1girl', 8601),
 ('long hair', 8066),
 ('smile', 7922),
 ('brown hair', 7342),
 ('black hair', 6908),
 ('1boy', 6875),
 ('multiple girls', 6252),
 ('blue eyes', 5859),
 ('multiple boys', 5818),
 ('blonde hair', 5441),
 ('solo', 5341),
 ('hat', 5126),
 ('open mouth', 4839),
 ('brown eyes', 4790),
 ('weapon', 4755),
 ('gloves', 4655),
 ('parody', 4276),
 ('2boys', 4028),
 ('2girls', 3825)]

In [10]:
'mecha' in all_tags

True

In [15]:
encode_tag = dict(zip(list(all_tags), range(nb_tags)))

In [12]:
from mangaki.utils.algo import fit_algo, get_dataset_backup

triplets = Rating.objects.values_list('user_id', 'work_id', 'choice')
dataset, algo = fit_algo('als', triplets)

Computing M: (2071 × 10086)
Chrono: fill and center matrix [3q, 2369ms]
Shapes (2071, 20) (20, 10086)
Chrono: factor matrix [3q, 8392ms]


In [13]:
import numpy as np

T = np.zeros((dataset.anonymized.nb_works, nb_tags))

In [18]:
for poster in tags:
    work_id = int(poster.replace('.jpg', ''))
    if work_id in dataset.encode_work:
        encoded_work_id = dataset.encode_work[work_id]
        for tag, weight in tags[poster]['general']:
            T[encoded_work_id][encode_tag[tag]] = weight

In [24]:
with open('../pickles/tags.pickle', 'wb') as f:
    np.save(f, T)